In [ ]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

In [ ]:
filepath = Path("C:/Users/igorp/Downloads/0.3_80.00_50.00_LVIDEAL_HEX20.xplt")
# filepath = Path("C:/Users/igornobrega/Downloads/0.3_80.00_50.00_LVIDEAL_HEX20.xplt")

# Create speckles

In [ ]:
lv = LV.from_file(filepath)
lv.identify_regions(LV_GEO_TYPES.IDEAL,
    apex_base_args={"ab_ql":0.03, "ab_qh":0.90},
    endo_epi_args={"threshold":85,})
lv.plot("mesh", 
        scalars=LV_MESH_DATA.SURFS,
        container="points",
        show_edges=True,
        categorical=True,
        split_sharp_edges=True,
        smooth_shading=False,
        pretty=False,
        vnodes=[
            (LV_VIRTUAL_NODES.BASE, "red"),
            (LV_VIRTUAL_NODES.APEX, "red")
            ],
        )

Create Longitudinal Speckles

In [ ]:
import logging

for i, a in enumerate(np.linspace(0, np.pi, 6, endpoint=False)):
    
    spk = lv.create_speckles(
        collection="long-6",
        group="endo",
        name=str(i),
        from_nodeset=LV_SURFS.ENDO,
        d=1.75,
        k=0.5,
        normal_to=[np.cos(a),np.sin(a),0.0],
        n_subsets=6,
        subsets_criteria="z2",
        t=0.0,
        kmin=0.125,
        kmax=0.95,
        log_level=logging.INFO,
    )

    spk = lv.create_speckles(
        collection="long-6",
        group="epi",
        name=str(i),
        from_nodeset=LV_SURFS.EPI,
        d=2.4,
        k=0.5,
        normal_to=[np.cos(a),np.sin(a),0.0],
        n_subsets=6,
        subsets_criteria="z2",
        t=0.0,
        kmin=0.125,
        kmax=0.95,
        log_level=logging.INFO,
    )

lv.set_region_from_speckles("long-6", spk_collection="long-6")
lv.plot(
    scalars="long-6", 
    categorical=True, 
    cmap="jet",
    vnodes=[
            (LV_VIRTUAL_NODES.BASE, "red"),
            (LV_VIRTUAL_NODES.APEX, "red")
            ],
    )

In [ ]:
lv.set_region_from_speckles("TEST", spk_name="0", spk_group="endo", spk_collection="long-6")
lv.plot(
    scalars="TEST", 
    categorical=True, 
    cmap="jet",
    vnodes=[
            (LV_VIRTUAL_NODES.BASE, "red"),
            (LV_VIRTUAL_NODES.APEX, "red")
            ],
    )

In [ ]:
import logging
names = ["subapex", "apex", "superapex", "submid", "mid", "supermid", "subbase", "base", "superbase"]

for i, a in enumerate(np.linspace(0.1, 1.0, len(names), endpoint=False)):
    
    spk = lv.create_speckles(
        collection="circ-6",
        group="endo",
        name=names[i],
        from_nodeset=LV_SURFS.ENDO,
        d=1.75,
        k=a,
        normal_to=[0.0, 0.0, 1.0],
        n_subsets=6,
        subsets_criteria="angles",
        t=0.0,
        kmin=-1.0,
        kmax=-1.0,
        log_level=logging.INFO,
    )

    spk = lv.create_speckles(
        collection="circ-6",
        group="epi",
        name=names[i],
        from_nodeset=LV_SURFS.EPI,
        d=1.75,
        k=a,
        normal_to=[0.0, 0.0, 1.0],
        n_subsets=6,
        subsets_criteria="angles",
        t=0.0,
        kmin=-1.0,
        kmax=-1.0,
        log_level=logging.INFO,
    )

lv.set_region_from_speckles("circ-6", spk_collection="circ-6")
lv.plot(
    scalars="circ-6", 
    categorical=True, 
    cmap="jet",
    vnodes=[
            (LV_VIRTUAL_NODES.BASE, "red"),
            (LV_VIRTUAL_NODES.APEX, "red")
            ],
    )

In [ ]:
lv.set_region_from_speckles("TEST", spk_name="base", spk_collection="circ-6")
lv.plot(
    scalars="TEST", 
    categorical=True, 
    cmap="Set2",
    vnodes=[
            (LV_VIRTUAL_NODES.BASE, "red"),
            (LV_VIRTUAL_NODES.APEX, "red")
            ],
    show_edges=True
    )

# Metrics

## Geo Metrics (formely geochars)

Volume:

In [ ]:
lv.volume()
lv.plot_metric(lv.STATES.VOLUME, kind="line")

Longitudinal Distances:

In [ ]:
lv.longitudinal_distances()
lv.plot_metric(lv.STATES.LONG_DISTS, kind="line", search_suffix={lv.REGIONS.ENDO, lv.REGIONS.EPI})

In [ ]:
lv.plot_metric("longitudinal_distance_2", kind="line")

In [ ]:
lv.plot_metric("longitudinal_distance_2", kind="line")

### Geo Metrics that do require speckles:

Radius (for each spk):

Expected values for radius at base: 

* EPI: 35
* ENDO: 25
* AVG: 30

In [ ]:
circ_spks = lv.get_speckles(spk_collection="circ-6")
lv.radius(circ_spks, recompute=True)
lv.plot_metric(lv.STATES.RADIUS, 
    from_ts=0.0)

Thickness

Expected: 10

In [ ]:
endo_circ_spks = lv.get_speckles(spk_name="base", spk_group="endo", spk_collection="circ-6")
epi_circ_spks = lv.get_speckles(spk_name="base", spk_group="epi", spk_collection="circ-6")

lv.thickness(endo_circ_spks, epi_circ_spks)
lv.plot_metric(lv.STATES.THICKNESS, from_ts=0.0, plot_infos={'group'})

Longitudinal Length:

In [ ]:
long_spks = lv.get_speckles(spk_collection="long-6")
lv.longitudinal_length(long_spks, recompute=True, reduce_by={"group", "name"},
                                           mfilter_ws=0,
                                           sfilter_ws=0,
                                           sfilter_or=0,
                                           )
lv.plot_metric(lv.STATES.LONG_LENGTH, 
    from_ts=0.0,
    plot_infos=["group", "name"])

Circumferential Length:

In [ ]:
circ_spks = lv.get_speckles(spk_collection="circ-6")
lv.circumferential_length(circ_spks, recompute=True, reduce_by={"group", "name"})
lv.plot_metric(lv.STATES.CIRC_LENGTH, 
    from_ts=0.0,
    plot_infos=["group", "name"])

Rotation:

In [ ]:
circ_spks = lv.get_speckles(spk_name="base", spk_collection="circ-6")
lv.rotation(circ_spks, recompute=True, check_orientation=False)
lv.plot_metric(lv.STATES.ROTATION, 
    from_ts=0.0,
    plot_infos=["group"])

## Clinical Metrics:

### Clinical metrics that do not require speckles

Ejection Fraction

In [ ]:
# lv.ejection_fraction()
# lv.plot_metric(lv.STATES.EF, 
#     from_ts=0.1,
#     plot_infos=["group"])

Longitudinal Shortening:

In [ ]:
# lv.longitudinal_shortening()
# lv.plot_metric(lv.STATES.LS, 
#     from_ts=0.1,
#     plot_infos=["group"])

### Clinical metrics that require speckles

Radial shortening


In [ ]:
circ_spks = lv.get_speckles(spk_collection="circ-6")
lv.radial_shortening(circ_spks, t_ed=0.0, recompute=True)
lv.plot_metric(lv.STATES.RS, 
    from_ts=0.0,
    plot_infos=["group"])

Wall thickening:

In [ ]:
endo_circ_spks = lv.get_speckles(spk_group="endo", spk_collection="circ-6")
epi_circ_spks = lv.get_speckles(spk_group="epi", spk_collection="circ-6")

lv.wall_thickening(endo_circ_spks, epi_circ_spks, t_ed=0.1, recompute=True)
lv.plot_metric(lv.STATES.WT, 
    from_ts=0.0,
    plot_infos=["group"])

Longitudinal strain:

In [ ]:
long_spks = lv.get_speckles(spk_collection="long-6")
lv.longitudinal_strain(long_spks, t_ed=0.1)
lv.plot_metric(lv.STATES.SL, 
    from_ts=0.0,
    plot_infos=["group"])

Circumferential Strain:

In [ ]:
circ_spks = lv.get_speckles(spk_collection="circ-6")
lv.circumferential_strain(circ_spks, t_ed=0.1)
lv.plot_metric(lv.STATES.SC, 
    from_ts=0.0,
    plot_infos=["group"])

Twist:

In [ ]:
apex_spks = lv.get_speckles(spk_name="apex", spk_collection="circ-6")
base_spks = lv.get_speckles(spk_name="base", spk_collection="circ-6")

lv.twist(apex_spks, base_spks, t_ed=0.1)
lv.plot_metric(lv.STATES.TW, 
    from_ts=0.0,
    plot_infos=["group"])

Torsion:

In [ ]:
apex_spks = lv.get_speckles(spk_name="apex", spk_collection="circ-6")
base_spks = lv.get_speckles(spk_name="base", spk_collection="circ-6")

lv.torsion(apex_spks, base_spks, t_ed=0.1)
lv.plot_metric(lv.STATES.TO, 
    from_ts=0.0,
    plot_infos=["group"])